In [196]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [197]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [198]:
len(words)

32033

In [199]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [200]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  
  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [201]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [202]:
C = torch.randn((27, 2))

In [203]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [204]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [205]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [206]:
h.shape

torch.Size([32, 100])

In [207]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [208]:
logits = h @ W2 + b2

In [209]:
logits.shape

torch.Size([32, 27])

In [210]:
counts = logits.exp()

In [211]:
prob = counts / counts.sum(1, keepdim=True)

In [212]:
prob.shape

torch.Size([32, 27])

In [213]:
prob[0].sum()

tensor(1.0000)

In [214]:
loss = -prob[torch.arange(32), Y].log().mean()

In [215]:
loss

tensor(17.8076)

In [216]:
# ------------ now made respectable :) ---------------

In [217]:
X.shape, Y.shape # dataset

(torch.Size([32, 3]), torch.Size([32]))

In [218]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [219]:
sum(p.nelement() for p in parameters)

3481

In [220]:
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27])
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.7697)

In [221]:
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27])
loss = F.cross_entropy(logits, Y)
loss

tensor(17.7697)

In [ ]:
# why cross_entropy

# first pytorch is using fused kernel

# second it can mathematically derive formulas to shorten backward pass, requiring less computations, 
# eg. tanh for micrograd, rather than multiple steps with exp approach

# it is well behaved, eg. taking exp(), manually will lead to floating point memory limitations
# explained below -

In [223]:
logits = torch.tensor([-5, -3, 0, 100])
counts = logits.exp()
probs = counts / counts.sum()
probs

tensor([0., 0., 0., nan])

In [224]:
counts

tensor([0.0067, 0.0498, 1.0000,    inf])

In [225]:
logits = torch.tensor([-5, -3, 0, 5])
counts = logits.exp()
probs = counts / counts.sum()
print(probs)

logits = torch.tensor([-5, -3, 0, 5]) - 10
counts = logits.exp()
probs = counts / counts.sum()
print(probs)

tensor([4.5079e-05, 3.3309e-04, 6.6903e-03, 9.9293e-01])
tensor([4.5079e-05, 3.3309e-04, 6.6903e-03, 9.9293e-01])


you can see the probs are same, below is the explaination for it

<img src="../public/manual softmax prevent overflow.png" style="width:600px" />

In [226]:
t = torch.tensor([-5, -3, 0, 5])
logits = t - max(t)
counts = logits.exp()
probs = counts / counts.sum()
print(probs)

tensor([4.5079e-05, 3.3309e-04, 6.6903e-03, 9.9293e-01])


In [ ]:
# the F.cross_entropy(logits, Y), already implements above